In [1]:
# Finding an optimal setup for longitudinal resolution
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks
import sys
sys.path.insert(0, "../../python")
from plotting import makefig
import eo_signal as eos
import currents as cp
def gen_fname(ctype, y0, tp, r0, method):
    y0s   = str(int(y0 * 1e9))
    tps   = str(int(tp * 1e15))
    r0s   = str(int(r0*1e3))
    dpath = "/home/keenan/Dropbox/CU/EOS-BPM/long_optimization/"
    npath = method + "/" + y0s + "/" + tps + "/"
    fname = ctype + "_" + str(int(r0 * 1e3)) + "mm"
    return dpath + npath, fname

In [11]:
def run_scan(ctype, ds, y0, tp, angles, r0, method, th = 0, plot = False):
    fpath, fname = gen_fname(ctype, y0, tp, r0, method)
    sig_diff = np.zeros((len(ds), len(angles)))
    max_sig  = np.zeros((len(ds), len(angles)))
    stn      = np.zeros((len(ds), len(angles)))
    wtd      = np.zeros((len(ds), len(angles)))
    for i in range(len(ds)):
        #print(i + 1, "of", len(ds))
        d = ds[i]
        for j in range(len(angles)):
            angle  = angles[j]
            I_peak = np.zeros(10)
            S_peak = np.zeros(10)
            max_S  = np.zeros(10)
            noise  = np.zeros(10)
            peaks  = np.zeros(10)
            for k in range(10):
                I, ti, sig, t_sig, gamma, t_gamma = eos.get_signal(k, ctype, d, y0, tp, angle, r0, method, th)
                I_peak[k], S_peak[k], noise[k], peaks[k] = eos.analyze_sig(I, sig, ti, t_sig, 0.4e-12)
                max_S[k] = max(sig)
            max_sig[i,j]   = np.nanmean(max_S)
            sig_diff[i, j] = np.nanmean(abs(I_peak - S_peak))
            stn[i,j]       = np.nanmean(noise)
            wtd[i,j]       = np.nanmean(peaks)
    return max_sig, sig_diff, stn, wtd, fpath, fname


def plot_study(max_sig, sig_diff, stn, wtd, method, study, fpath, fname, r0, save = False):

    # Create figure
    if study == "error":
        fig, ax = makefig(xlab = r'$\theta$ [deg.]', ylab = r'Average Error [$\mu$m]')
        data = sig_diff
    elif study == "peak":
        fig, ax = makefig(xlab = r'$\theta$ [deg.]', ylab = r'Average Peak Signal [AU]')
        data = max_sig
    elif study == "stn":
        fig, ax = makefig(xlab = r'$\theta$ [deg.]', ylab = r'Average Witness STN')
        data = stn
    elif study == "wtd":
        fig, ax = makefig(xlab = r'$\theta$ [deg.]', ylab = 'Average Witness to Drive Ratio')
        data = wtd
        
    for i in range(len(ds)):
        lab = str(int(ds[i]*1e6 + 0.5)) + r' $\mu$m'
        plt.plot(angles, data[i,:], 'o', label = lab)
    ax.legend()
    if save:
        plt.savefig(fpath + fname)
    plt.show()
        

In [35]:
ctype  = "ZnTe"
ds     = np.array([25, 50, 100, 200]) * 1e-6
y0     = 800e-9
tp     = 30e-15
angles = [5, 15, 30, 45, 60]
r0     = 1e-3
method = "near"
th     = 90 * np.pi / 180

In [36]:
max_sig, sig_diff, stn, wtd, fpath, fname = run_scan(ctype, ds, y0, tp, angles, r0, method, th)
%matplotlib tk
plot_study(max_sig, sig_diff, stn, wtd, method, 'error', fpath, fname, r0)
plot_study(max_sig, sig_diff, stn, wtd, method, 'peak', fpath, fname, r0)
plot_study(max_sig, sig_diff, stn, wtd, method, 'stn', fpath, fname, r0)
plot_study(max_sig, sig_diff, stn, wtd, method, 'wtd', fpath, fname, r0)